In [ ]:
import psycopg2
import pandas as pds
from sqlalchemy import create_engine
import datetime
from typing import List, Tuple
import random

## Extracción de los datos sobre las tablas

In [ ]:
DATABASE = "proyecto"
PORT="5432"
HOST="localhost"
USERNAME="alexndf"
PASSWORD="postgrespw123"

alchemy_engine   = create_engine(f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')
db_connection = alchemy_engine.connect()

table_data_query = """
    SELECT t.table_name,
        pg_catalog.obj_description(pgc.oid, 'pg_class') AS "description"
    FROM information_schema.tables t
        INNER JOIN pg_catalog.pg_class pgc ON t.table_name = pgc.relname
    WHERE t.table_type = 'BASE TABLE'
        AND t.table_schema = 'public'
"""
table_data = pds.read_sql(table_data_query, db_connection)

column_data_query = """
    SELECT c.table_name,
        c.column_name,
        c.data_type,
        pgd.description
    FROM pg_catalog.pg_statio_all_tables AS st
        INNER JOIN pg_catalog.pg_description pgd ON (pgd.objoid = st.relid)
        INNER JOIN information_schema.columns c ON (
            pgd.objsubid = c.ordinal_position
            AND c.table_schema = st.schemaname
            AND c.table_name = st.relname
        );
"""
column_data = pds.read_sql(column_data_query, db_connection)


db_connection.close()

In [ ]:
display(table_data)
display(column_data)

## Formateo de los datos

In [ ]:
def get_latex_p_col(pct):
    return "p{"+f"\\dimexpr {pct:0.2f}\\textwidth-2\\tabcolsep"+"}"

In [ ]:
table_table = pds.DataFrame()
table_table["Nombre"] = [
    r"\texttt{" + str(s).replace("_", r"\_") + "}" for s in table_data["table_name"]
]
table_table["Descripción"] = table_data["description"]
table_data_latex_str = (
    table_table.style.hide(axis="index")
    .to_latex()
    .replace("tabular", "longtable")
    .replace(
        "{ll}",
        "{|" + get_latex_p_col(0.35) + "|" + get_latex_p_col(0.65) + "|}\n\hline",
    )
    .replace("\\\\\n", "\\\\ \\hline\n")
)
print(table_data_latex_str)


In [ ]:
column_data

In [ ]:
table_names = list(sorted(table_data["table_name"]))
column_tables = []
column_data_latex_strs = []
for tn in table_names:
    current_column_data = column_data[column_data["table_name"] == tn]

    current_column_table = pds.DataFrame()
    current_column_table["Nombre"] = [
        r"\texttt{" + str(s).replace("_", r"\_") + "}"
        for s in current_column_data["column_name"]
    ]
    current_column_table["Tipo"] = [
        r"\texttt{" + str(s).replace("_", r"\_") + "}"
        for s in current_column_data["data_type"]
    ]
    current_column_table["Descripción"] = list(current_column_data["description"])
    column_tables.append(current_column_table)
    current_column_table_latex_str = (
        r"Columnas de \texttt{"
        + tn.replace("_", r"\_")
        + "}:\n"
        + (
            current_column_table.style.hide(axis="index")
            .to_latex()
            .replace("tabular", "longtable")
            .replace(
                "{lll}",
                "{|"
                + get_latex_p_col(0.34)
                + "|"
                + get_latex_p_col(0.22)
                + "|"
                + get_latex_p_col(0.44)
                + "|}\n\hline",
            )
            .replace("\\\\\n", "\\\\ \\hline\n")
        )
    )
    column_data_latex_strs.append(current_column_table_latex_str)


In [ ]:
print("\n".join(column_data_latex_strs))